In [ ]:
import numpy as np
import torch
from sklearn.model_selection import KFold
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
class EEGDataset(Dataset):
    def __init__(self, data_path):
        mat_data = loadmat(data_path)
        data = mat_data['data']
        self.data = data[:, 0:272]
        self.labels = data[:, 272:273]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return torch.from_numpy(sample).float(), torch.tensor(label).long()  

In [ ]:
dataset = EEGDataset(data_path="./your_dataset")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## NN
1 hidden layer

In [ ]:
# 定义神经网络
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.1):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)  
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)  
        out = self.relu(out)
        out = self.dropout(out)  
        out = self.fc2(out)
        return out


In [ ]:
dataset = EEGDataset(data_path="./your_dataset")

np.random.seed(0)
torch.manual_seed(0)

kf = KFold(n_splits=10, shuffle=True)

accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):

    train_dataset = Subset(dataset, train_index)
    test_dataset = Subset(dataset, test_index)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model = SimpleNN(input_size=272, hidden_size=100, num_classes=3, dropout_rate=0.1).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    
    for epoch in range(30):  
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

    model.eval()
    y_pred, y_test = [], []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            y_pred += predicted.tolist()
            y_test += label.tolist()

    
    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))
    

metrics1 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}

for metric, values in metrics1.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")

## DNN1 5 layers
5 hidden layers batchnorm dropout

In [ ]:
class ComplexNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.5):
        super(ComplexNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.bn4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.bn5 = nn.BatchNorm1d(hidden_size)
        self.dropout5 = nn.Dropout(dropout_rate)

        self.fc6 = nn.Linear(hidden_size, num_classes)

        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)
        x = self.relu(self.bn5(self.fc5(x)))
        x = self.dropout5(x)
        x = self.fc6(x)
        return x


In [ ]:
dataset = EEGDataset(data_path="./your_dataset")

np.random.seed(0)
torch.manual_seed(0)

kf = KFold(n_splits=10, shuffle=True)

accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):

    train_dataset = Subset(dataset, train_index)
    test_dataset = Subset(dataset, test_index)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model = ComplexNN(input_size=272, hidden_size=100, num_classes=3, dropout_rate=0.1).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(33):  
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

    
    model.eval()
    y_pred, y_test = [], []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            y_pred += predicted.tolist()
            y_test += label.tolist()

    
    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))

metrics2 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}


for metric, values in metrics2.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")




## DNN2 residual 5 layers

In [ ]:
class ResidualComplexNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.5):
        super(ResidualComplexNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.bn4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        
        self.fc5 = nn.Linear(2 * hidden_size, hidden_size)
        self.bn5 = nn.BatchNorm1d(hidden_size)
        self.dropout5 = nn.Dropout(dropout_rate)

        self.fc6 = nn.Linear(hidden_size, num_classes)

        self.relu = nn.ReLU()
    
    def forward(self, x):
        x1 = self.relu(self.bn1(self.fc1(x)))
        x1 = self.dropout1(x1)

        x2 = self.relu(self.bn2(self.fc2(x1)))
        x2 = self.dropout2(x2)

        x3 = self.relu(self.bn3(self.fc3(x2)))
        x3 = self.dropout3(x3)

        x4 = self.relu(self.bn4(self.fc4(x3)))
        x4 = self.dropout4(x4)

        
        x4 = torch.cat((x1, x4), dim=1)

        x5 = self.relu(self.bn5(self.fc5(x4)))
        x5 = self.dropout5(x5)

        x6 = self.fc6(x5)
        return x6


In [ ]:
dataset = EEGDataset(data_path="./your_dataset")

np.random.seed(0)
torch.manual_seed(0)

kf = KFold(n_splits=10, shuffle=True)

accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):

    train_dataset = Subset(dataset, train_index)
    test_dataset = Subset(dataset, test_index)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model = ResidualComplexNN(input_size=272, hidden_size=100, num_classes=3, dropout_rate=0.1).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(30): 
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

    model.eval()
    y_pred, y_test = [], []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            y_pred += predicted.tolist()
            y_test += label.tolist()

    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))

metrics3 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}


for metric, values in metrics3.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")


## DNN3 residual 9 layers

In [ ]:
class DualResidualNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.5):
        super(DualResidualNN, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.bn4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        
        self.fc5 = nn.Linear(2 * hidden_size, hidden_size)
        self.bn5 = nn.BatchNorm1d(hidden_size)
        self.dropout5 = nn.Dropout(dropout_rate)

        
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.bn6 = nn.BatchNorm1d(hidden_size)
        self.dropout6 = nn.Dropout(dropout_rate)

        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.bn7 = nn.BatchNorm1d(hidden_size)
        self.dropout7 = nn.Dropout(dropout_rate)

        self.fc8 = nn.Linear(hidden_size, hidden_size)
        self.bn8 = nn.BatchNorm1d(hidden_size)
        self.dropout8 = nn.Dropout(dropout_rate)

        
        self.fc9 = nn.Linear(2 * hidden_size, hidden_size)
        self.bn9 = nn.BatchNorm1d(hidden_size)
        self.dropout9 = nn.Dropout(dropout_rate)

        self.fc_out = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        
        residual1 = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(residual1)

        
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)

        
        x = torch.cat((residual1, x), dim=1)
        x = self.relu(self.bn5(self.fc5(x)))
        x = self.dropout5(x)

        residual2 = x

        x = self.relu(self.bn6(self.fc6(x)))
        x = self.dropout6(x)
        x = self.relu(self.bn7(self.fc7(x)))
        x = self.dropout7(x)
        x = self.relu(self.bn8(self.fc8(x)))
        x = self.dropout8(x)

        x = torch.cat((residual2, x), dim=1)
        x = self.relu(self.bn9(self.fc9(x)))
        x = self.dropout9(x)

        x = self.fc_out(x)

        return x


In [ ]:
dataset = EEGDataset(data_path="./your_dataset")

np.random.seed(0)
torch.manual_seed(0)

kf = KFold(n_splits=10, shuffle=True)


accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):

    train_dataset = Subset(dataset, train_index)
    test_dataset = Subset(dataset, test_index)


    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


    model = DualResidualNN(input_size=272, hidden_size=100, num_classes=3, dropout_rate=0.1).to(device)


    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)


    for epoch in range(35):  
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

    
    model.eval()
    y_pred, y_test = [], []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            y_pred += predicted.tolist()
            y_test += label.tolist()

    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))

metrics4 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}


for metric, values in metrics4.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")


